In [1]:
import zipfile

'''with zipfile.ZipFile("indian-sign-board-image-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")'''

with zipfile.ZipFile("archive.zip", "r") as zip_ref:
    zip_ref.extractall("dataset1")

print('Done')

Done


In [2]:
import os
import pandas as pd

# Paths (adjust if necessary)
LABELS_CSV = "dataset1/Indian-Traffic Sign-Dataset/traffic_sign.csv"   # Contains columns: ClassId, Name
IMAGES_ROOT = "dataset1/Indian-Traffic Sign-Dataset/Images"      # Subfolders named by ClassId
OUTPUT_CSV = "dataset1/processed_dataset1.csv"

# 1. Load ClassId → Name mapping
mapping_df = pd.read_csv(LABELS_CSV, dtype={"ClassId": str})
# Create dict: {'0': 'Speed_Limit_30', '1': 'Stop', ...}
id_to_name = dict(zip(mapping_df["ClassId"], mapping_df["Name"]))

# 2. Traverse image folders and build records
records = []
for class_id, name in id_to_name.items():
    folder = os.path.join(IMAGES_ROOT, class_id)
    if not os.path.isdir(folder):
        continue
    for fname in os.listdir(folder):
        if not fname.lower().endswith((".jpg", ".png")):
            continue
        # filename relative to IMAGES_ROOT
        rel_path = os.path.join(class_id, fname)
        records.append({"filename": rel_path, "label": name})

# 3. Save to processed_dataset.csv
df = pd.DataFrame(records, columns=["filename", "label"])
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Wrote {len(df)} entries to {OUTPUT_CSV}")
print("Classes found:", df["label"].unique())

✅ Wrote 13971 entries to dataset1/processed_dataset1.csv
Classes found: ['Give way' 'No entry' 'One-way traffic' 'No vehicles in both directions'
 'No entry for cycles' 'No entry for goods vehicles'
 'No entry for pedestrians' 'No entry for bullock carts'
 'No entry for hand carts' 'No entry for motor vehicles' 'Height limit'
 'Weight limit' 'Axle weight limit' 'Length limit' 'No left turn'
 'No right turn' 'No overtaking' 'Maximum speed limit (90 km/h)'
 'Maximum speed limit (110 km/h)' 'Horn prohibited' 'No parking'
 'No stopping' 'Turn left' 'Turn right' 'Steep descent' 'Steep ascent'
 'Narrow road' 'Narrow bridge' 'Unprotected quay' 'Road hump' 'Dip'
 'Loose gravel' 'Falling rocks' 'Cattle' 'Crossroads' 'Side road junction'
 'Oblique side road junction' 'T-junction' 'Y-junction'
 'Staggered side road junction' 'Roundabout'
 'Guarded level crossing ahead' 'Unguarded level crossing ahead'
 'Level crossing countdown marker' 'Parking' 'Bus stop' 'First aid post'
 'Telephone' 'Filling s

In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Config
IMG_SIZE = 128
CSV_PATH = "dataset1/processed_dataset1.csv"
IMG_DIR = "dataset1/Indian-Traffic Sign-Dataset/Images"
BATCH_SIZE = 16
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CSV
df = pd.read_csv(CSV_PATH)
df = df.drop_duplicates(subset="filename")

# Encode labels
le = LabelEncoder()
df["label_enc"] = le.fit_transform(df["label"])
num_classes = len(le.classes_)

# Train/Test Split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Dataset Class
class TrafficSignDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.loc[idx, 'filename'])
        image = Image.open(img_path).convert('RGB')
        label = int(self.df.loc[idx, 'label_enc'])

        if self.transform:
            image = self.transform(image)

        return image, label

# Transforms
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# Datasets and Dataloaders
train_dataset = TrafficSignDataset(train_df, IMG_DIR, transform=transform)
val_dataset = TrafficSignDataset(val_df, IMG_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# CNN Model
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * (IMG_SIZE // 8) * (IMG_SIZE // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Initialize Model
model = CNNModel(num_classes).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"🧠 Epoch {epoch+1} - Loss: {running_loss:.4f} | Accuracy: {train_acc:.2f}%")

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
print(f"✅ Validation Accuracy: {val_acc:.2f}%")

Epoch 1/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 699/699 [05:20<00:00,  2.18it/s]


🧠 Epoch 1 - Loss: 1679.1290 | Accuracy: 34.02%


Epoch 2/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 699/699 [03:31<00:00,  3.30it/s]


🧠 Epoch 2 - Loss: 893.5116 | Accuracy: 60.93%


Epoch 3/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 699/699 [03:32<00:00,  3.29it/s]


...

🧠 Epoch 8 - Loss: 336.1721 | Accuracy: 83.60%


Epoch 9/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 699/699 [03:31<00:00,  3.30it/s]


🧠 Epoch 9 - Loss: 309.5003 | Accuracy: 85.09%


Epoch 10/10: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 699/699 [03:25<00:00,  3.41it/s]


🧠 Epoch 10 - Loss: 290.3202 | Accuracy: 86.05%
✅ Validation Accuracy: 84.11%


In [4]:
# Save the trained entire model
torch.save(model, "model.pth")

# or Save the trained model weights
torch.save(model.state_dict(), "model_weights.pth")
print('Done')

Done
